<img src="../img/GTK_Logo_Social Icon.jpg" width=175 align="right" />

# Worksheet 1: Network Analytics - Answers

In this worksheet, you will try out some of the techniques used for network analysis using Python.  It should take approximate 20-30 minutes to complete. 

## Import the Libraries

This worksheet will use the following modules:
* `dnsparse`: https://dnspython.readthedocs.io/en/latest/
* `python-whois`: https://pypi.org/project/python-whois/
* `ip2geotools`: https://github.com/tomas-net/ip2geotools


In [20]:
# Run me first
import whois
import csv
import dns.resolver, dns.reversename, dns.query, dns.zone
from ip2geotools.databases.noncommercial import DbIpCity

## Challege 1: Geolocating an Email Server
In this challenge, you have a domain and you will use network analytic tools to geolocate the mail server.  Let's use cnn.com as a target.  Feel free to pick another target.

### Step 1:  Get the MX records
For the first step, execute a DNS query to retrieve the MX records for your domain.  To do this, use `dns.resolver.resolve` from `dnsparse` and get the MX records for that domain. Print out the MX records.  Remember there may be more than one.

In [8]:
mxrecords = dns.resolver.resolve('crtc.gc.ca', 'MX')

for mxrecord in mxrecords:
    print('MX Record: ',mxrecord)

MX Record:  10 crtc-gc-ca.mail.protection.outlook.com.


### Step 2:  Get the MX Servers' A Records

If this worked, you should now have the mail server(s) for your domain.  Next, let's attempt to find the IP addresses associated with these servers.  We will accomplish that by executing another DNS query, but this time, we'll look for the `A` record.  Remember, we want the `A` record for the MX server, not the domain.

In [9]:
arecords = dns.resolver.resolve('crtc-gc-ca.mail.protection.outlook.com', 'A')
for arecord in arecords:
    response = DbIpCity.get(str(arecord), api_key='free')
    print(f"{arecord}: City: {response.city}, Country: {response.country}, Region: {response.region}")

104.47.75.228: City: Quebec City, Country: CA, Region: Quebec
104.47.75.164: City: Toronto, Country: CA, Region: Ontario


### Step 3:  Geolocate the MX Servers
Using the module `ip2geotools`, let's attempt to geolocate these IP addresses. The snippet below will geolocate an IP address:

```python
response = DbIpCity.get(str(<ip address>), api_key='free')
```

Once you have the `response` you can extract the various components of that response.

In [ ]:
# Your code here...

## Challenge 2:  Finding Vulnerable Domains
In this challenge, we will look for domains in the Alexa 1M that do not have SPF/DKIM or DMARC enabled.

What you'll need to do in this exercise is:

1.  Execute a DNS query to retrieve the `TXT` records for each domain. 
2.  For each domain, examine each `TXT` record to see whether it is a valid SPF, DKIM or DMARC.  
3.  If the domain has all three enabled, print that the domain is secure.  If not, print which one is missing.


**HINT:  Not all domains in the data are valid or will have TXT records. Be sure to enclose them in a try/catch block.**


In [10]:
# Get the domains from the CSV into a list
domains = []
with open('../data/bottom100.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        domains.append(row[1])

In [11]:
# Your code here ...
for domain in domains:
    try:
        txt_records = dns.resolver.resolve(domain, 'TXT')
        found_spf = False
        found_dkim = False
        found_dmarc = False
        
        # Iterate over the text records to find the relevant ones
        for txt in txt_records:
            txt_record = txt.to_text()
            
            if "v=spf1" in txt_record:
                found_spf = True
            elif "v=DKIM" in txt_record:
                found_dkim = True
            elif "v=DMARC" in txt_record:
                found_dmarc = True
            
        if found_spf and found_dkim and found_dkim:
            print(f"{domain} is secure.")
        else:
            warning_string = f"{domain} is missing: "
            if not found_spf:
                warning_string += "SPF "
            elif not found_dkim:
                warning_string += "DKIM "
            elif not found_dmarc:
                warning_string += "DMARC."
            
            print(warning_string)
        
    except:
        print(f"{domain} not found.")
        continue

zurf.nl is missing: DKIM 
totalhealth.co.uk is missing: DKIM 
iporno.com.br is missing: DKIM 
erodojinhustle.com not found.
kagi1109948.com not found.
desafiobtc.com is missing: DKIM 
xdownloadfilmgratis.id not found.
surecall.com is missing: DKIM 
findnew2upgrades.download not found.
hanaif.re.kr not found.
realestatesales.gov is missing: DKIM 
karakuriya.com is missing: DKIM 
lenovo-service.ir not found.
xiaoyangren.net not found.
zlatibor.rs is missing: DKIM 
agronet.gov.co is missing: SPF 
agrodataperu.com is missing: DKIM 
energyfromthevacuum.com is missing: SPF 
riverrides.com is missing: DKIM 
veganfitness.net not found.
rre6oei2g00781t8.cf not found.
enseignementeducation.fr is missing: SPF 
gimje.go.kr is missing: DKIM 
japan-fans.com is missing: SPF 
potomaccrossfit.com is missing: DKIM 
cricketmatchbettingtips.com not found.
holdingtotruth.com is missing: DKIM 
essie.de is missing: DKIM 
ldcvip.com not found.
nokiarevolution.com is missing: DKIM 
mkkssmkjabar.or.id not found

## Challenge 3: Verifying an SPF Record and Reconnaissance

For this challenge, we will be doing some network reconnaissance and verifying an SPF record. First, execute a DNS query for all `TXT` records for the domain `foxnews.com`. 

What services and verifications do you see?

In [51]:
txt_records = dns.resolver.resolve("foxnews.com", 'TXT')
for r in txt_records:
    print(r)

"v=spf1 ip4:66.230.193.40 ip4:66.230.193.2 ip4:208.84.65.98 ip4:208.86.201.96 include:spf-00195501.pphosted.com include:spf.protection.outlook.com include:amazonses.com include:mail.zendesk.com include:_spf.google.com -all"
"MS=ms40284671"
"265947818-2009536"
"qRWnq9UOByGW6DnvW8qZ4scp8GbkRYG4bsmSOyP+dzlIB+XXQtkNbpBK3qVrJ8E7YT83Bk33z5CPO1L2KlH/mA=="
"adobe-idp-site-verification=10f6011913207e944a026129f59881b0cb1078801a8c11229c6e95615eb28070"
"_globalsign-domain-verification=BahbT-Pu-HaLP9bBimZ0MGe-4CPc4Z_MXqKNUajBmF"
"google-site-verification=3LvSKyvB7eXlZQtS_7fwgd0cMKh6zBBzo-g9hhEVu7k"
"MS=ms71309079"
"bppko2051pbcn9bvdualgach7h"


In the SPF record, you should see a few IP addresses.  Pick one that starts with `208`.  We're going to do a reverse DNS lookup on this IP address.  A reverse DNS lookup is exactly what it sounds like.  You start with an IP address and the DNS server will return entries that have an `A` record which includes that IP address. 

To execute the reverse DNS lookup, use the following command:

```python
dns.reversename.from_address(ip)
```

In [60]:
addrs = dns.reversename.from_address("208.84.65.98")

# Increase the timeout
my_resolver = dns.resolver.Resolver()

# 8.8.8.8 is Google's public DNS server and 1.1.1.1 is Cloudflare
my_resolver.nameservers = ['1.1.1.1', '8.8.8.8', '8.8.4.4']


for ip in my_resolver.resolve(addrs,"PTR"):
    print(str(ip))

When you print out the results of this call, you should get a hostname.  Compare that with the original query and execute a DNS query of MX records for this domain.  Is there a DNS MX entry for that server?

In [61]:
txt_records = my_resolver.resolve("foxnews.com", "MX")
for r in txt_records:
    print(r)

10 mxa-00195501.gslb.pphosted.com.
10 mxb-00195501.gslb.pphosted.com.
